<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-sign-language-recognition-in-video/blob/main/NN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import math
import sys
import cv2
import io

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from sklearn.model_selection import train_test_split

from google.colab import drive

In [ ]:
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/SU2/data"

Mounted at /content/drive


In [ ]:
# !git clone https://github.com/Strojove-uceni/23206-final-sign-language-recognition-in-video.git
!git clone https://github.com/jiri99/SU_sign-language.git

Cloning into '23206-final-sign-language-recognition-in-video'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 164 (delta 13), reused 0 (delta 0), pack-reused 135
Receiving objects: 100% (164/164), 2.59 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [ ]:
repo_path = "/content/23206-final-sign-language-recognition-in-video/"
sys.path.append(os.path.abspath(repo_path))

In [ ]:
# from buffer import ParquetData
# from CNN_draft import NpyFolderDataset

In [ ]:
# selected_landmark_indices = [33, 133, 159, 263, 46, 70, 4, 454, 234, 10, 338, 297, 332, 61, 291, 0, 78, 14, 317,
#                              152, 155, 337, 299, 333, 69, 104, 68, 398]

df_train = pd.read_csv(path + "/train_mod.csv", sep=",")
df_train.head()

# data_load = ParquetData()
# data_load.read_all(path, df_train, selected_landmark_indices)

In [ ]:
# tensor_load = NpyFolderDataset(path)
path_tensor = path + "/tensors"

npy_files = []
labels = []
class_to_idx = {}
for idx, class_name in enumerate(sorted(os.listdir(path_tensor))):
    class_dir = os.path.join(path_tensor, class_name)
    if os.path.isdir(class_dir):
        class_to_idx[class_name] = idx
        for file in os.listdir(class_dir):
            if file.endswith('.npy'):
                npy_files.append(os.path.join(class_dir, file))
                labels.append(idx)

In [ ]:
stacked_data = []

for npy_file in npy_files:
  sample = np.load(npy_file)
  sample = np.expand_dims(sample.astype(np.float32), axis=3)
  stacked_data.append(sample)

In [ ]:
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(stacked_data, labels, test_size=0.2, random_state=42)

In [ ]:
tensor_data = np.stack(X_train_raw, axis=0)
X_train = tensor_data.reshape((540, 420, 70, 70, 1))
tensor_data = np.stack(X_test_raw, axis=0)
X_test = tensor_data.reshape((60, 420, 70, 70, 1))

In [ ]:
# Y_train = np.stack(y_train_raw, axis=0)
one_hot_encoded_train = pd.get_dummies(y_train_raw)
Y_train = tf.convert_to_tensor(one_hot_encoded_train, dtype=tf.float32)
one_hot_encoded_test = pd.get_dummies(y_test_raw)
Y_test = tf.convert_to_tensor(one_hot_encoded_test, dtype=tf.float32)
# Y_test = np.stack(y_test_raw, axis=0)

In [ ]:
X_train.shape

(540, 420, 70, 70, 1)

In [ ]:
Y_train.shape

TensorShape([540, 4])

In [ ]:
# stacked_data = []
# stacked_sign = []

# for key_sign in list(data_load.data.keys()):
#   for key_participiant in list(data_load.data[key_sign].keys()):
#     stacked_sign.append(key_sign)
#     stacked_data.append(data_load.data[key_sign][key_participiant].tensor)

# tensor_data = np.stack(stacked_data, axis=0)
# X_train = tensor_data.reshape((len(stacked_sign), 420, 70, 70, 1))
# stack_sign = np.stack(stacked_sign, axis=0)
# one_hot_encoded = pd.get_dummies(stack_sign)
# Y_train = tf.convert_to_tensor(one_hot_encoded, dtype=tf.float32)

In [ ]:
frame_height = 70
frame_width = 70
channels = 1
# classes_num = len(list(data_load.data.keys()))
classes_num = 10
sequence_length = 420

# CNN model for spatial feature extraction
cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(frame_height, frame_width, channels)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten()
])

# Final model
model = Sequential([
    TimeDistributed(cnn, input_shape=(sequence_length, frame_height, frame_width, channels)),
    LSTM(64, return_sequences=False),
    Dense(64, activation='relu'),
    Dense(classes_num, activation='softmax')  # Assuming 10 classes for the classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 420, 6272)         92672     
 ributed)                                                        
                                                                 
 lstm (LSTM)                 (None, 64)                1622272   
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 1719364 (6.56 MB)
Trainable params: 1719364 (6.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Now you can fit this model on your preprocessed data.
# X_train would be your video data with shape (num_samples, sequence_length, frame_height, frame_width, channels)
# Y_train would be your labels
model.fit(X_train, Y_train, epochs=10, batch_size=4)

Epoch 1/10
135/135 [==============================] - 23s 73ms/step - loss: 1.3949 - accuracy: 0.2259
Epoch 2/10
135/135 [==============================] - 10s 72ms/step - loss: 1.3872 - accuracy: 0.2481
Epoch 3/10
135/135 [==============================] - 10s 73ms/step - loss: 1.3872 - accuracy: 0.2796
Epoch 4/10
135/135 [==============================] - 10s 73ms/step - loss: 1.3919 - accuracy: 0.2685
Epoch 5/10
135/135 [==============================] - 10s 73ms/step - loss: 1.2525 - accuracy: 0.3815
Epoch 6/10
135/135 [==============================] - 10s 72ms/step - loss: 0.9670 - accuracy: 0.5500
Epoch 7/10
135/135 [==============================] - 10s 72ms/step - loss: 0.7857 - accuracy: 0.6593
Epoch 8/10
135/135 [==============================] - 10s 72ms/step - loss: 0.6407 - accuracy: 0.7222
Epoch 9/10
135/135 [==============================] - 10s 72ms/step - loss: 0.5368 - accuracy: 0.7667
Epoch 10/10
135/135 [==============================] - 10s 73ms/step - loss: 0.430

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, Y_test, batch_size=4)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

15/15 [==============================] - 1s 28ms/step - loss: 0.7827 - accuracy: 0.7333
Test Loss: 0.7827401757240295
Test Accuracy: 0.7333333492279053
